In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
import pandas as pd
from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Define dataset path
dataset_path = "/content/drive/My Drive/ExDark"
augmented_data_folder = "Augmented Data"
os.makedirs(augmented_data_folder, exist_ok=True)

In [4]:
from google.colab import files
import zipfile

In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [6]:
# Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [7]:
import random

# Define the number of augmented images required (30% more)
num_original_images = len(full_dataset)
num_augmented_images = int(0.3 * num_original_images)

# Define augmentation transformations
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of flipping
    transforms.RandomRotation(15),  # Rotate by ±15 degrees
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])

augmented_images = []
augmented_labels = []

# Randomly select images to augment until we reach the 30% increase
for i in range(num_augmented_images):
    original_idx = random.randint(0, num_original_images - 1)  # Pick a random image
    image, label = full_dataset[original_idx]

    # Convert tensor to PIL for augmentation
    image = transforms.ToPILImage()(image)

    # Apply augmentation
    augmented_image = augmentation_transforms(image)

    # Save augmented image
    augmented_image_path = os.path.join(augmented_data_folder, f"augmented_{i}.jpg")
    transforms.ToPILImage()(augmented_image).save(augmented_image_path)

    augmented_images.append(augmented_image_path)
    augmented_labels.append(full_dataset.classes[label])

In [8]:
# Save augmented labels in a CSV file
pd.DataFrame({"image": augmented_images, "label": augmented_labels}).to_csv("augmented_labels.csv", index=False)

In [9]:
# Define ResNet-52 (Modified ResNet-50)
class ResNet52(nn.Module):
    def __init__(self, num_classes):
        super(ResNet52, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.layer4[2] = nn.Sequential(
            self.model.layer4[2],  # Existing Bottleneck
            nn.Conv2d(2048, 2048, kernel_size=3, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU()
        )
        self.model.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        return self.model(x)

In [10]:
# Initialize model
num_classes = len(full_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet52(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 175MB/s]


In [11]:
# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True)
batch_size = 32
metrics = {"accuracy": [], "precision": [], "recall": [], "f1": []}

for fold, (train_idx, val_idx) in enumerate(kf.split(full_dataset)):
    print(f"Fold {fold+1}...")
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Train model
    for epoch in range(5):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate model
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    metrics["accuracy"].append(accuracy)
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1"].append(f1)

    print(f"Fold {fold+1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# Save trained model
torch.save(model.state_dict(), "resnet52_model.pt")
torch.save(model, "resnet52_model.h5")
print("Model saved successfully.")

# Save final results
df_results = pd.DataFrame(metrics)
df_results.to_csv("final_results.csv", index=False)
print("Final results saved.")

Fold 1...
Fold 1 - Accuracy: 0.4481, Precision: 0.4724, Recall: 0.4481, F1: 0.4354
Fold 2...
Fold 2 - Accuracy: 0.5173, Precision: 0.5618, Recall: 0.5173, F1: 0.5105
Fold 3...
Fold 3 - Accuracy: 0.6463, Precision: 0.7034, Recall: 0.6463, F1: 0.6210
Fold 4...
Fold 4 - Accuracy: 0.8084, Precision: 0.8288, Recall: 0.8084, F1: 0.8061
Fold 5...
Fold 5 - Accuracy: 0.8845, Precision: 0.8959, Recall: 0.8845, F1: 0.8833
Model saved successfully.
Final results saved.


In [12]:
import shutil
import os

# Define paths
augmented_data_path = "/content/drive/My Drive/Augmented Data"
zip_file_path = "/content/drive/My Drive/Augmented_Data.zip"

# Ensure the directory exists before zipping
if os.path.exists(augmented_data_path):
    # Zip the folder
    shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', augmented_data_path)
    print("✅ Augmented data has been zipped successfully.")
else:
    print("❌ Error: Augmented Data folder not found!")


❌ Error: Augmented Data folder not found!


In [13]:
import shutil
import os

# Define new directory inside /content/ (Google Colab's working directory)
new_directory = "/content/drive/My Drive/my_saved_files"
os.makedirs(new_directory, exist_ok=True)

# Define paths
augmented_data_path = "/content/drive/My Drive/Augmented Data"  # Ensure this exists
model_path_pt = "/content/drive/My Drive/resnet52_model.pt"
model_path_h5 = "/content/drive/My Drive/resnet52_model.h5"
labels_path = "/content/drive/My Drive/augmented_labels.csv"

# Zip the augmented data folder
zip_augmented_data = f"{new_directory}/Augmented_Data.zip"
if os.path.exists(augmented_data_path):
    shutil.make_archive(zip_augmented_data.replace(".zip", ""), 'zip', augmented_data_path)
    print("✅ Augmented data has been zipped successfully.")
else:
    print("❌ Error: Augmented Data folder not found!")

# Move model and labels files to the new directory
if os.path.exists(model_path_pt):
    shutil.move(model_path_pt, f"{new_directory}/resnet52_model.pt")
if os.path.exists(model_path_h5):
    shutil.move(model_path_h5, f"{new_directory}/resnet52_model.h5")
if os.path.exists(labels_path):
    shutil.move(labels_path, f"{new_directory}/augmented_labels.csv")

print(f"✅ All files are saved in: {new_directory}. Ready for download.")

# Auto-download the files
from google.colab import files

# Download model files and CSV
for file_name in ["resnet52_model.pt", "resnet52_model.h5", "augmented_labels.csv", "Augmented_Data.zip"]:
    file_path = f"{new_directory}/{file_name}"
    if os.path.exists(file_path):
        files.download(file_path)
    else:
        print(f"❌ Warning: {file_name} not found.")


❌ Error: Augmented Data folder not found!
✅ All files are saved in: /content/drive/My Drive/my_saved_files. Ready for download.
❌ Warning: resnet52_model.pt not found.
❌ Warning: resnet52_model.h5 not found.
❌ Warning: augmented_labels.csv not found.
❌ Warning: Augmented_Data.zip not found.


In [17]:
import os

augmented_data_path = "/content/Augmented Data"
if os.path.exists(augmented_data_path):
    print("✅ Augmented Data folder exists.")
else:
    print("❌ Augmented Data folder NOT found! Check if data was generated correctly.")


✅ Augmented Data folder exists.


In [18]:
import shutil

# Define paths
zip_file_path = "/content/download/my_saved_files/Augmented_Data.zip"
augmented_data_path = "/content/Augmented Data"

# Try creating the zip file
if os.path.exists(augmented_data_path):
    os.makedirs("/content/download/my_saved_files", exist_ok=True)  # Create folder if not exist
    shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', augmented_data_path)
    print("✅ Zip file created successfully.")
else:
    print("❌ Error: Augmented Data folder not found!")


✅ Zip file created successfully.


In [19]:
if os.path.exists(zip_file_path):
    print("✅ Zip file exists, ready to move.")
else:
    print("❌ Zip file NOT found! There is an issue with zipping.")


✅ Zip file exists, ready to move.


In [20]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define Google Drive path
drive_folder = "/content/drive/My Drive/Colab Saved Files"
os.makedirs(drive_folder, exist_ok=True)

# Move files only if they exist
for file_name in ["Augmented_Data.zip", "resnet52_model.pt", "resnet52_model.h5", "augmented_labels.csv"]:
    file_path = f"/content/download/my_saved_files/{file_name}"
    if os.path.exists(file_path):
        shutil.move(file_path, f"{drive_folder}/{file_name}")
        print(f"✅ {file_name} moved to Google Drive.")
    else:
        print(f"❌ Warning: {file_name} not found. Skipping.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Augmented_Data.zip moved to Google Drive.
❌ Warning: resnet52_model.pt not found. Skipping.
❌ Warning: resnet52_model.h5 not found. Skipping.
❌ Warning: augmented_labels.csv not found. Skipping.


In [21]:
import os

model_path_pt = "/content/resnet52_model.pt"

if os.path.exists(model_path_pt):
    print("✅ Model file (.pt) exists.")
else:
    print("❌ Model file (.pt) NOT found! Make sure you saved it correctly.")


✅ Model file (.pt) exists.


In [22]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define Google Drive path
drive_folder = "/content/drive/My Drive/Colab Saved Files"
os.makedirs(drive_folder, exist_ok=True)

# Move model file only if it exists
model_file_path = "/content/resnet52_model.pt"
if os.path.exists(model_file_path):
    shutil.move(model_file_path, f"{drive_folder}/resnet52_model.pt")
    print("✅ Model file (.pt) moved to Google Drive.")
else:
    print("❌ Warning: Model file (.pt) not found. Skipping.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model file (.pt) moved to Google Drive.
